# TechXchange Japan 2024: さわってみようベクトル・データベース watsonx.dataでRAG体験


生成AIの回答精度を向上させるために、自社内のデータを活用してみましょう！<br>
ベクトル・データベース + 大規模言語モデル（LLM）で構成されるRAGのアーキテクチャーを使えば、自社内の情報で生成AIチャットボットが作成できます。<br>
当ハンズオンでは「TechXchange Japan 2024」に関するデータを使ってRAGを構成し、「TechXchange Japan 2024」に関することを教えてくれるチャットボットを作成します。<br>
(以下ベクトル・データベースはベクトルDBと表記します。)<br>


具体的には大規模言語モデル（LLM）を使用したアプリケーション開発のためのオープンソース・オーケストレーション・フレームワーク[LangChain](https://python.langchain.com/docs/introduction/)を使って、wastosonx.dataのベクトルDB **Milvus**に「TechXchange Japan 2024」に関するデータをロードし、watson.aiで提供されているLLMを使用してRAGを構成し、「TechXchange Japan 2024」のことを回答してくれるChatbotを作ってみます。

ハンズオンは以下の順序で実行します:

1. Excelをベクトル化してベクトルDB Milvusに入れよう！
2. **ベクトルDB Milvusに入ったデータで類似検索してみよう!** [**当notebook**]
3. **ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!** [**当notebook**]
4. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!

**当notebookは2と3を実施します。**<br>

このハンズオンのガイドは[**こちら**](https://ibm.biz/20241127-rag-handson)にあります。


## 2. ベクトルDB Milvusに入ったデータで類似検索してみよう! 


<img  src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/notebook02_01.jpg?raw=true">
<br>
<br>

**実行は　セルを選択(クリック)して**

- **Windows： Ctrl + Enter** 
- **Mac: ⌘ (command) + Enter 　または Ctrl + Enter**

**実行して次のセルを選択は　セルを選択(クリック)して**

- **Windows： Shift + Enter** 
- **Mac: Shift + Enter**

セルの左側に[*]が表示されている場合は実行中です。<br>
[1]のように数字が入っている場合は、実行が完了しています。

### 1. 必要なライブラリーのインストール

In [ ]:
print("pip install start")
!pip install -Uq 'ibm-watsonx-ai>=1.1.15'
!pip install -Uq 'langchain>=0.3.3'
!pip install -Uq 'langchain-ibm>=0.3.1'
!pip install -Uq 'langchain-milvus>=0.1.6'
!pip install -Uq 'langchain-community>=0.3.2'
!pip install -Uq 'pymilvus>=2.4.8'
print("pip install completed")

### 2. apikeyの設定 

- 実行するとテキスト入力ボックスが表示されるので、事前に取得したapikeyをに入れてEnter Keyを押してください

- ハンズオン環境で実行の場合は、`IBM Cloud Service API key`を入力してください。　よくわからない場合は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/01_techzone_use_environments.md)の2-3を参照してください。

In [ ]:
import getpass
apikey = getpass.getpass("apikeyを入れてEnter Keyを押してください:")

### 3. Milvus接続情報の設定

実行すると順に以下の2つのテキスト入力ボックスが表示されるので、事前に取得したMilvus接続情報の値を入力してください。
- milvus GRPC ホストを入れてEnter Keyを押してください →  GRPC ホストの値を入力  

- milvus GRPC ポートを入れてEnter Keyを押してください　→ GRPC ポートの値をを入力


Milvus接続情報の詳細取得手順は[こちら](https://github.com/IBM/japan-technology/tree/main/techxchange/2024-watsonx-handson-1/watsonx_data_get_milvus_info.md) を参照

簡略な接続手順は以下です:
- watsonx.dataの画面を開く
- ナビゲーションメニューから「インフラストラクチャー・マネージャー」を選択
- サービス「Milvus」をクリック
- タイプの下の「接続の詳細を見る」をクリック
- GRPC ホストの値とGRPC ポートの値を取得
  
collection名は`techxchange_line_data`としています。

In [ ]:
milvus_host=input("milvus GRPC ホストを入れてEnter Keyを押してください: ")
milvus_port=input("milvus GRPC ポートを入れてEnter Keyを押してください: ")

In [ ]:
# Milvus接続情報パラメータののセット
my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{apikey}'
}

my_collection =  'techxchange_line_data'

### 4. watsonx.ai Project idの設定

Watson Studioで実行する場合は、このノートブックが実行されるプロジェクトからProject idを取得します。
Watson Studio以外で実行する場合は、Project idを入力してください。

**Hint**: `project_id` はプロジェクトを表示し、管理タブから `project_id` を取得可能です.

In [ ]:
import os
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Project idを入力してください (入力後enter): ")

### 5.watsonx.aiのAuthentication用のエンドポイントのURLの設定

Waston Machine Learningのインスタンスを作成したリージョンで決まります。
https://ibm.github.io/watson-machine-learning-sdk/setup_cloud.html#authentication　より

- Dallas: https://us-south.ml.cloud.ibm.com
- London: https://eu-gb.ml.cloud.ibm.com
- Frankfurt: https://eu-de.ml.cloud.ibm.com
- Tokyo: https://jp-tok.ml.cloud.ibm.com

今回はダラスのWaston Machine Learningのインスタンスを使っているので`https://us-south.ml.cloud.ibm.com`を使います。

In [ ]:
watsonx_url = "https://us-south.ml.cloud.ibm.com" #watsonx.aiのAuthentication用のエンドポイントのURL

### 6. 必要ライブラリーのImport

In [ ]:
import pandas as pd
from langchain.schema.document import Document
import json
from langchain_milvus import Milvus
import os
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

### 7. Embeddingモデルの取得
ベクトル化した時と同じモデル`intfloat/multilingual-e5-large`を使います<br>
- https://huggingface.co/intfloat/multilingual-e5-large
- https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-embed.html?context=wx&locale=ja#multilingual-e5-large

LangChainで使用できるwatsonx.aiのEmbeddingモデル`intfloat/multilingual-e5-large`があるので、今回はこちらを使用します:
- https://python.langchain.com/docs/integrations/text_embedding/ibm_watsonx/
  
---

>尚、`intfloat/multilingual-e5-large`はオープンソースで公開されているので、watsonx.aiのEmbeddingモデルを使用しなくとも、ローカルにダウンロードすることで使用可能です。
その場合のコードはこちらです(今回は使用しません)

>```python
>from langchain_huggingface import HuggingFaceEmbeddings
>from tqdm.autonotebook import tqdm
>embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

>```
> 上記コード実行時の`TqdmExperimentalWarning`のWarningは無視でよいです

In [ ]:
# watsonx.aiのEmbeddingモデル取得
embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 512,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

embeddings = WatsonxEmbeddings(
    model_id="intfloat/multilingual-e5-large",
    url=watsonx_url ,
    apikey=apikey,
    project_id=project_id
    )

### 8.  ベクトルDB Milvusに接続

In [ ]:
from langchain_milvus import Milvus

vector_store = Milvus(
    embeddings,
    connection_args =my_connection_args,
    collection_name = my_collection
)

### 9.  挿入データの確認

Milvus DBにロードした内容をDataFrameにダンプして表示させます。

In [ ]:
from pymilvus import MilvusClient
import pandas as pd

pk_list=vector_store.get_pks(expr="pk > 0")

client = MilvusClient(uri=my_connection_args['uri'], token=my_connection_args['token'])

res = client.get(
    collection_name= my_collection,
    ids=pk_list
)

for i, milvus_rec in enumerate(res):
    vecter_data = milvus_rec['vector']
    res[i]['vector']= "["+", ".join(map(str, vecter_data))+"]"
    res[i]['pk']= str(milvus_rec['pk'])


df_s = pd.DataFrame.from_dict(res).reindex(columns=['ID', 'Category', 'text', 'pk', 'vector'])
df_s

### 10. ベクトルDBでテキスト類似検索してみます

参考: LangChain ドキュメント: [Vector stores → Search ](https://python.langchain.com/docs/concepts/vectorstores/#search)

基本、類似度が高い順でリストされます。いろいろなオプションで検索してみます。

#### 10-1. similarity_search: オプションなし デフォルト

In [ ]:
# オプションなし
query = "IBM TechXchange Japanとは?"
docs = vector_store.similarity_search(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-2. similarity_search: 結果の取得数をkで指定(デフォルトは4)

In [ ]:
# 結果の取得数をkで指定(デフォルトは4)

docs = vector_store.similarity_search(query, k=10)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-3. similarity_search_with_score: 類似度のスコアも一緒に出力
- 類似度のスコアも一緒に出してみます
- スコアはコサイン類似度の場合は、1に近いほど類似度が高いです。

In [ ]:
# 類似度のスコアも一緒に出してみます
# スコアはコサイン類似度の場合は、1に近いほど類似度が高いです。

docs = vector_store.similarity_search_with_score(query, k=10)
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 10-4. similarity_search_with_score exprオプション: 'Category': '概要'　でフィルター

In [ ]:
# 'Category': '概要'　でフィルターしてみます
docs = vector_store.similarity_search_with_score(query, k=10, expr="Category=='概要'")
for doc, score in docs:
    print({"score": score, "content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

### これで「2. ベクトルDB Milvusに入ったデータで類似検索してみよう!」は完了です。<br>
### 次の「3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!」に進みます
<br>

<br>

---

## 3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!

<img  src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/notebook03_01.jpg?raw=true">
<br>
<br>


### 1.  watsonx.ai LLMの取得

今回は[mixtral-8x7b-instruct-v01](https://www.ibm.com/docs/ja/watsonx/saas?topic=solutions-supported-foundation-models#mixtral-8x7b-instruct-v01)を使用します。

In [ ]:
# watsonx.ai LLM: 必要なライブラリーのImport
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_ibm import WatsonxLLM

# 使用するLLMのパラメータ
generate_params = {
    GenParams.MAX_NEW_TOKENS: 16384,
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.DECODING_METHOD: "greedy",
    GenParams.REPETITION_PENALTY: 1
}

# LangChainで使うllm
custom_llm = WatsonxLLM(
    model_id="mistralai/mixtral-8x7b-instruct-v01",
    url=watsonx_url,
    apikey=apikey,
    project_id=project_id,
    params=generate_params,
)


### 2. ベクトルデータベースからLangChain Retrieverの作成

参考: LangChain ドキュメント: [Retrievers](https://python.langchain.com/docs/concepts/retrievers/)

Retrieverは検索のレスポンスを返すlangchainのInterfaceです。<br>
主にベクトルストアから作成可能ですが、Wikipedia検索などからでも作成できます。<br>
Retrieverを使うとLLMと組み合わせたRAG構成がLangChainで簡単にできます。<br>
「2. ベクトルDB Milvusに入ったデータで類似検索してみよう! 」で使用したvector storeとの違いは以下です:
- vector storeは1つ種類のベクトルDBに特化したもの。今回はMilvusのvector storeを使用しています。
- RetrieverはベクトルDBおよび他の検索システムに汎用的に使用できるインターフェースであり、query文字列を入力としLangchainのDocumentのListを出力する単純なものです。Chainを構成することができるため、RAG構成でよく使用されます。

ここではいろいろな条件のRetrieverをベクトルDBから作成してみます。

#### 2-1. as_retriever: オプションなし デフォルト構成

In [ ]:
# オプションなし デフォルト構成
query = "IBM TechXchange Japanとは?"
retriever = vector_store.as_retriever()

# retriever を使用したベクトルDBの類似検索
docs = retriever.invoke(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")

#### 2-2.  as_retriever: 結果の取得数をkで指定(デフォルトは4)

In [ ]:
# 結果の取得数をkで指定(デフォルトは4)
retriever = vector_store.as_retriever(search_kwargs={"k": 10})
docs = retriever.invoke(query)

for doc in docs:
    print({"content": doc.page_content[0:100], "metadata": doc.metadata} )
    print("---------")


#### 2-3.   類似スコアの閾値を設定し、その閾値以上のスコアを持つ文書のみを返す
langchain_milvus.Mivusにはsearch_type="similarity_score_threshold"の設定がありません。<br>
参考: [Similarity score threshold retrieval](https://python.langchain.com/docs/how_to/vectorstore_retriever/#similarity-score-threshold-retrieval) はlangchain_milvus v2.4.8では使えません

自分で「閾値以上の類似スコアを持つ文書のみを返す」CustomRetrieverを作成すれば可能です。

##### 2-3-1. CustomRetrieverの作成
閾値以上の類似スコアを持つ文書のみを返す」CustomRetrieverを作成

In [ ]:
# 閾値以上の類似スコアを持つ文書のみを返す」CustomRetrieverを作成
# asyncの方は省略
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.schema.document import Document
from langchain.callbacks.manager import (
    CallbackManagerForRetrieverRun,
)
from typing import List

class CustomRetriever(VectorStoreRetriever):
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        top_k = self.search_kwargs.get("k", 4)
        docs_and_similarities = self.vectorstore.similarity_search_with_score(query, k=top_k)
                
        threshold = self.search_kwargs.get("score_threshold", 0)
       
        return [doc for doc, score in docs_and_similarities if score >= threshold and score <= 1]

##### 2-3-2. CustomRetriever使用: 類似スコア 0.85以上のもの、　上位最大5件

In [ ]:
# 類似スコア 0.85以上のもの、　上位最大5件
retriever = CustomRetriever(
    vectorstore = vector_store,
    search_kwargs={"score_threshold": 0.85, "k":5},
)
docs = retriever.invoke(query)
# print(docs)
for doc in docs:
    print({"content": doc.page_content, "metadata": doc.metadata} )
    print("---------")

### 3. ベクトル・データベース ＋ 生成AI(LLM)のRAG構成で質問に回答してみよう

プロンプトを準備してベクトル・データベースの検索結果を使って質問に回答してみましょう。<br>
ベクトル・データベースの検索結果取得には7で作成したRetrieverを使います。<br>
watsonx.ai LLMは1で取得しています。<br>
LangChainのchainという機能を使って、組み合わせます。<br>

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# retrieverは単純なデフォルトのものにします（変えてもOK）
retriever = vector_store.as_retriever()

# Prompt Templateを設定します。これはmixtral-8x7b-instruct-v01に合わせて作成してあります
template = """<s> [INST] 
あなたは親切で、礼儀正しく、誠実なアシスタントです。常に安全を保ちながら、できるだけユーザーの役に立つように詳しく回答してください。
回答には、有害、非倫理的、⼈種差別的、性差別的、有毒、危険、または違法なコンテンツを含めてはいけません。回答は社会的に偏⾒がなく、本質的に前向きなものであることを確認してください。
質問が意味をなさない場合、または事実に⼀貫性がない場合は、正しくないことに答えるのではなく、その理由を説明してください。質問の答えがわからない場合は、誤った情報を共有しないでください。
questionに答えるために、以下のcontextを使用し必ず日本語でanswerを作成してください。
必ず⽇本語の文章で回答してください。知ったかぶりをしないでください。
回答を書くときは、context内の単語をできるだけ使⽤してください。context中に質問に対する回答がない場合は、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。「文書中に質問に対する回答が明記されていません。」と回答した場合、そこで回答を終わりにしてください。
contextの内容がブランクの場合、「文書中に質問に対する回答が明記されていません。」とだけ回答してください。
セッションについて回答する場合は、タイトル、開始時刻、終了時刻、会場、概要、レベルを回答してください。回答が200文字以上の場合、回答はなるべく箇条書きを含めてわかりやすく回答してください。
[/INST]
</s>
<s> [INST] 質問が質問の文章ではなく意味がわからない場合[/INST]もう少し詳しく説明していただけますか？</s>

context: {context}
question: {question}
answer: 
"""

# Prompt Templateを作成します
rag_prompt = PromptTemplate.from_template(template)

# chainを作成します

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | custom_llm
)

ではTechXchangeに関する質問をしてみましょう！<br>
`rag_chain.invoke(<質問文>)`　でRAGの仕組みでベクトル検索の結果から生成AIが文章を作成し回答します。

お好みの質問文で置き換えるか、コメント`#` を外して質問してみてください。

In [ ]:
query="RAGに関するセッションの詳細を教えてください"
# query="watsonx.dataに関するセッションの詳細を教えてください"
# query="TechXchangeについて概要を教えてください"
# query="量子コンピューター関連のセッションを教えてください"
print(rag_chain.invoke(query))

---
試しに情報をベクトルDBに入れていない、TechXchangeとは関係ない一般的な質問をしてみましょう！

In [ ]:
query="日本の首相は誰ですか"
print(rag_chain.invoke(query))

ベクトルDBに入れていない情報は回答しないようにプロンプトで指示しているので、回答しません。これで不正確な回答が防げます。<br>
ただしLLMによってそのような指示を書いても、無視するものもあったり、書き方で回答が変わったりするので、テストでプロンプトを調整したり、必要に応じて不要な回答をなくす後処理を追加してください。


---
### これで「3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!」は完了です。<br>


#### Notebookを保存する場合は、右上の保存アイコンをクリックして保存してください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/save_notebook.jpg?raw=true">
<br>
<br>

#### プロジェクトの画面に戻る場合は、右上のプロジェクト名をクリックしてください。

- <img width="400" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/return_to_project.jpg?raw=true">
<br>
<br>

#### Notebookを開いたままでプロジェクトの画面を表示するには、上部のプロジェクト名を右クリックし、「新しいタブで開く」でプロジェクトを新しいタブで開いてください。

- <img width="500" alt="" src="https://github.com/IBM/japan-technology/blob/main/techxchange/2024-watsonx-handson-1/images/open_project_from_notebook.jpg?raw=true">
<br>
<br>

#### 次の「4. ベクトル・データベース Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!」に進んでください。